In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q accelerate open3d pillow laspy
!pip install -q OpenEXR

In [ ]:
# Install Apple's Depth Pro and dependencies
!pip install git+https://github.com/apple/ml-depth-pro.git --quiet
!pip install open3d --quiet
!pip install timm --quiet

# Download the checkpoint (if not handled automatically by the code)
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O depth_pro.pt
!pip install "numpy<2.0"


In [ ]:
import os

# ==============================================================================
# CONFIGURATION
# ==============================================================================

# --- MODEL SELECTION ---
# Options: "depth_anything_metric", "depth_anything_non_metric", "depth_pro"
# 
# MODEL COMPARISON (depth4 dataset, least_squares scaling):
#   Depth Pro:              RMSE=0.055m, d1=100% (RECOMMENDED)
#   Depth Anything Metric:  RMSE=0.084m, d1=99.9%
#   Depth Pro is 35% more accurate!
MODEL_TYPE = "depth_pro"

# --- DATASET SELECTION ---
# Available datasets in ./data/:
#   - depth4: Room scene with edit.png (recommended)
#   - depth5: Same as depth4
#   - depth_gt_finder: Flat wall calibration test
#   - mrq2: Movie Render Queue output (different format!)
DATASET = "depth4"

INPUT_FOLDER = f"./data/{DATASET}"
OUTPUT_FOLDER = "./output"

# --- FILE NAMING ---
# Adjust these based on your dataset and UE export format
if DATASET == "mrq2":
    GT_IMG = "Seq.FinalImage.0000.exr"
    GT_DEPTH_IMG = "Seq.FinalImageMovieRenderQueue_WorldDepth.0000.exr"
    GT_TO_CENTIMETERS = 1.0  # MRQ WorldDepth is already in centimeters
else:
    GT_IMG = "HighresScreenshot00000.exr"  # RGB image
    GT_DEPTH_IMG = "HighresScreenshot00000_SceneDepth.exr"  # Depth file
    # VERIFIED: SceneDepth * 10000 = centimeters (flat wall test confirmed)
    GT_TO_CENTIMETERS = 10000.0

# Optional: Use WorldUnits file instead (already in meters)
# GT_DEPTH_IMG = "HighresScreenshot00000_SceneDepthWorldUnits.exr"
# GT_TO_CENTIMETERS = 100.0  # WorldUnits * 100 = centimeters

# Optional decorated/edited image for comparison
DECORATED_IMG = "edit.png" if os.path.exists(f"{INPUT_FOLDER}/edit.png") else None

DEPTH_PRO_CHECKPOINT = "depth_pro.pt"

# Computed paths
IMG_PATH = f"{INPUT_FOLDER}/{GT_IMG}"
GT_DEPTH_PATH = f"{INPUT_FOLDER}/{GT_DEPTH_IMG}"
DECORATED_PATH = f"{INPUT_FOLDER}/{DECORATED_IMG}" if DECORATED_IMG else None

OUTPUT_LAS = f"{OUTPUT_FOLDER}/room.las"
OUTPUT_LAS_DECORATED = f"{OUTPUT_FOLDER}/room_decorated.las"

# Create output directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- MODEL CONFIGURATIONS ---
MODEL_CONFIGS = {
    "depth_anything_metric": {
        "hf_id": "depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf",
        "is_metric": True,
        "type": "transformers"
    },
    "depth_anything_non_metric": {
        "hf_id": "depth-anything/Depth-Anything-V2-Large-hf",
        "is_metric": False,
        "type": "transformers"
    },
    "depth_pro": {
        "checkpoint": DEPTH_PRO_CHECKPOINT,
        "is_metric": True,
        "type": "depth_pro"
    }
}

# --- SCALING METHOD ---
# Analysis results (depth4 dataset):
#   "none":          Unusable (RMSE > 0.5m for both models)
#   "median":        Good (RMSE ~0.07-0.14m)
#   "least_squares": BEST (RMSE ~0.05-0.08m)
SCALING_METHOD = "least_squares"

# --- CAMERA ---
CAMERA_FOV = 90.0  # Horizontal field of view in degrees (verified for UE camera)

# --- FILTERING ---
MIN_DEPTH = 0.1   # Minimum depth in meters
MAX_DEPTH = 50.0  # Maximum depth in meters

# --- POINT CLOUD CLEANING ---
CLEAN_POINT_CLOUD = False  # Set to True to filter depth edges
EDGE_THRESHOLD = 1.5       # Gradient threshold for edge detection

# ==============================================================================
# PRINT CONFIGURATION
# ==============================================================================
print(f"Configuration:")
print(f"  Dataset:    {DATASET}")
print(f"  Model:      {MODEL_TYPE}")
print(f"  Scaling:    {SCALING_METHOD}")
print(f"  GT->cm:     {GT_TO_CENTIMETERS}")
print(f"  Camera FOV: {CAMERA_FOV} deg")
print(f"  Cleaning:   {'Enabled' if CLEAN_POINT_CLOUD else 'Disabled'}")
print(f"\nFiles:")
print(f"  RGB:        {IMG_PATH}")
print(f"  GT Depth:   {GT_DEPTH_PATH}")
print(f"  Decorated:  {DECORATED_PATH}")
print(f"  Output:     {OUTPUT_FOLDER}/")

In [ ]:
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
import OpenEXR
import Imath
import os

# Model-specific imports
if MODEL_CONFIGS[MODEL_TYPE]["type"] == "transformers":
    from transformers import AutoImageProcessor, AutoModelForDepthEstimation
elif MODEL_CONFIGS[MODEL_TYPE]["type"] == "depth_pro":
    import depth_pro
    import shutil

import laspy  # For LAS output

In [ ]:
def load_exr_rgb(path):
    """Load RGB channels from EXR file"""
    exr_file = OpenEXR.InputFile(path)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    
    FLOAT = Imath.PixelType(Imath.PixelType.FLOAT)
    img_data = []
    for c in ['R', 'G', 'B']:
        channel_str = exr_file.channel(c, FLOAT)
        channel = np.frombuffer(channel_str, dtype=np.float32).reshape(height, width)
        img_data.append(channel)
    
    img = np.stack(img_data, axis=-1)
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    return Image.fromarray(img)


def load_exr_depth(path):
    """Load depth channel from EXR file safely checking for Z vs R"""
    exr_file = OpenEXR.InputFile(path)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    
    # Check channels: Prefer 'Z' (Planar) or 'SceneDepth' over 'R'
    channels = header['channels'].keys()
    if 'Z' in channels:
        print(f"  [EXR] Using 'Z' channel (Planar) for {os.path.basename(path)}")
        chan_name = 'Z'
    elif 'SceneDepth' in channels:
        print(f"  [EXR] Using 'SceneDepth' channel for {os.path.basename(path)}")
        chan_name = 'SceneDepth'
    else:
        print(f"  [EXR] Warning: Using 'R' channel. It is planar distance.")
        chan_name = 'R'

    FLOAT = Imath.PixelType(Imath.PixelType.FLOAT)
    channel_str = exr_file.channel(chan_name, FLOAT)
    
    depth = np.frombuffer(channel_str, dtype=np.float32).reshape(height, width).copy()
    
    # Sanity check for infinite/negative values
    depth[depth == np.inf] = 0
    depth[depth < 0] = 0
    return depth


def load_image(path):
    """Load image from EXR, PNG, or JPG"""
    ext = os.path.splitext(path)[1].lower()
    
    if ext == '.exr':
        return load_exr_rgb(path)
    elif ext in ['.png', '.jpg', '.jpeg']:
        return Image.open(path).convert("RGB")
    else:
        raise ValueError(f"Unsupported file format: {ext}")


def align_depth(pred_depth, gt_depth, method="median"):
    """Align predicted depth to ground truth using median, least squares, or none"""
    valid_mask = (gt_depth > 0) & (~np.isnan(gt_depth)) & (pred_depth > 0)
    gt_valid = gt_depth[valid_mask]
    pred_valid = pred_depth[valid_mask]
    
    if method == "none":
        # Crucial for Metric models (Depth Pro / Depth Anything V2 Metric)
        print("Alignment: None (Metric Mode - keeping raw values)")
        scale = 1.0
        shift = 0.0
        aligned = pred_depth
    elif method == "median":
        scale = np.median(gt_valid) / np.median(pred_valid)
        shift = 0.0
        aligned = pred_depth * scale
        print(f"Alignment: Median scaling (x{scale:.4f})")
    elif method == "least_squares":
        A = np.vstack([pred_valid, np.ones(len(pred_valid))]).T
        scale, shift = np.linalg.lstsq(A, gt_valid, rcond=None)[0]
        aligned = (pred_depth * scale) + shift
        print(f"Alignment: Least squares (x{scale:.4f} + {shift:.4f})")
    else:
        raise ValueError(f"Unknown scaling method: {method}")
    
    return aligned, scale, shift, valid_mask


def compute_metrics(pred_aligned, gt_depth, valid_mask):
    """Compute depth estimation metrics: RMSE, MAE, relative error, delta accuracy, log RMSE"""
    gt = gt_depth[valid_mask]
    pred = pred_aligned[valid_mask]
    
    # Standard Errors
    abs_diff = np.abs(gt - pred)
    mae = np.mean(abs_diff)
    rmse = np.sqrt(np.mean(abs_diff ** 2))
    
    # Relative Error
    rel_error = np.mean(abs_diff / gt) * 100
    
    # Accuracy Thresholds (Delta metrics)
    thresh = np.maximum((gt / (pred + 1e-6)), (pred / (gt + 1e-6)))
    delta1 = (thresh < 1.25).mean()
    delta2 = (thresh < 1.25 ** 2).mean()
    delta3 = (thresh < 1.25 ** 3).mean()
    
    # Log RMSE (Penalizes errors at close range more heavily)
    rmse_log = np.sqrt(np.mean((np.log(gt + 1e-6) - np.log(pred + 1e-6))**2))
    
    return rmse, mae, rel_error, delta1, rmse_log


print("Utility functions loaded")

In [ ]:
# model loading

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device.upper()}")

config = MODEL_CONFIGS[MODEL_TYPE]

if config["type"] == "transformers":
    print(f"Loading {MODEL_TYPE}...")
    processor = AutoImageProcessor.from_pretrained(config["hf_id"])
    model = AutoModelForDepthEstimation.from_pretrained(config["hf_id"]).to(device)
    print(f"✓ Model loaded: {config['hf_id']}")
    
elif config["type"] == "depth_pro":
    print(f"Loading Depth Pro...")
    os.makedirs("checkpoints", exist_ok=True)
    shutil.copy(config["checkpoint"], "checkpoints/depth_pro.pt")
    model, transform = depth_pro.create_model_and_transforms(
        device=device, 
        precision=torch.float16
    )
    model.eval()
    print("✓ Depth Pro loaded")

In [ ]:
print("Loading images...")
image = load_image(IMG_PATH)
gt_depth = load_exr_depth(GT_DEPTH_PATH)
h_gt, w_gt = gt_depth.shape

if config["type"] == "transformers":
    print("Running depth estimation...")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        pred_depth = model(**inputs).predicted_depth.squeeze().cpu().numpy()

    print(f"gt shape: {gt_depth.shape}")
    print(f"pred shape: {pred_depth.shape}")
    
    # Resize to match GT
    if pred_depth.shape == gt_depth.shape:
        print("shapes align - skipping resize")
    else:
        pred_depth = cv2.resize(pred_depth, (w_gt, h_gt), interpolation=cv2.INTER_LINEAR)
        print("pred_depth resized!")
    
    # Invert if non-metric model (disparity → depth)
    if not config["is_metric"]:
        print("⚠️ Non-metric model - inverting disparity to depth")
        pred_depth = 1.0 / (pred_depth + 1e-6)

elif config["type"] == "depth_pro":
    print("Running Depth Pro inference...")
    image_tensor = transform(image)
    with torch.no_grad():
        prediction = model.infer(image_tensor, f_px=None)
    
    pred_depth = prediction["depth"].squeeze().cpu().numpy()
    
    # Resize to match GT
    pred_depth = cv2.resize(pred_depth, (w_gt, h_gt), interpolation=cv2.INTER_LINEAR)

print(f"✓ Depth estimated (shape: {pred_depth.shape})")



# ← NEW: Run on decorated image if provided
decorated_depth = None
if DECORATED_PATH:
    decorated_image = load_image(DECORATED_PATH)
    w_dec, h_dec = decorated_image.size  # ← Keep native resolution
    if config["type"] == "transformers":
        inputs_dec = processor(images=decorated_image, return_tensors="pt").to(device)
        with torch.no_grad():
            decorated_depth = model(**inputs_dec).predicted_depth.squeeze().cpu().numpy()
        decorated_depth = cv2.resize(decorated_depth, (w_dec, h_dec), interpolation=cv2.INTER_LINEAR)
        if not config["is_metric"]:
            decorated_depth = 1.0 / (decorated_depth + 1e-6)
    elif config["type"] == "depth_pro":
        image_tensor_dec = transform(decorated_image)
        with torch.no_grad():
            prediction_dec = model.infer(image_tensor_dec, f_px=None)
        decorated_depth = prediction_dec["depth"].squeeze().cpu().numpy()
        decorated_depth = cv2.resize(decorated_depth, (w_dec, h_dec), interpolation=cv2.INTER_LINEAR)
    print(f"✓ Decorated depth estimated (shape: {decorated_depth.shape})")

In [ ]:
pred_aligned, scale, shift, valid_mask = align_depth(pred_depth, gt_depth, SCALING_METHOD)#############
#rmse, mae, rel_error = compute_metrics(pred_aligned, gt_depth, valid_mask)
rmse, mae, rel_error, delta1, rmse_log = compute_metrics(pred_aligned, gt_depth, valid_mask)

print(f"\n{'='*50}")
print(f"RESULTS ({MODEL_TYPE.upper()} - {SCALING_METHOD})")
print(f"{'='*50}")
print(f"  Scale:      {scale:.4f}")
if SCALING_METHOD == "least_squares":
    print(f"  Shift:      {shift:.4f}")
print(f"  RMSE:       {rmse:.6f}")
print(f"  MAE:        {mae:.6f}")
print(f"  Rel Error:  {rel_error:.4f}%\n")

print(f"  delta1:     {delta1:.4f}%")
print(f"  RMSE log:   {rmse_log:.6f}")
print(f"{'='*50}")


In [ ]:
# PLOTS

vmax = np.percentile(gt_depth[valid_mask], 95)
gt_viz = np.clip(gt_depth, 0, vmax)
pred_viz = np.clip(pred_aligned, 0, vmax)
error_viz = np.clip(np.abs(gt_depth - pred_aligned), 0, mae * 3)
error_viz[~valid_mask] = 0

plt.figure(figsize=(20, 5))

plt.subplot(141)
plt.imshow(image)
plt.title("Input RGB")
plt.axis('off')

plt.subplot(142)
plt.imshow(gt_viz, cmap='magma', vmin=0, vmax=vmax)
plt.title("Ground Truth Depth")
plt.colorbar(shrink=0.8)
plt.axis('off')

plt.subplot(143)
plt.imshow(pred_viz, cmap='magma', vmin=0, vmax=vmax)
plt.title(f"Predicted Depth (×{scale:.5f})")
plt.colorbar(shrink=0.8)
plt.axis('off')

plt.subplot(144)
plt.imshow(error_viz, cmap='inferno')
plt.title(f"Absolute Error (MAE: {mae:.6f})")
plt.colorbar(shrink=0.8)
plt.axis('off')

plt.tight_layout()
plt.show()


# ← NEW: Show decorated depth if available
if decorated_depth is not None:
    decorated_aligned = decorated_depth * scale
    decorated_viz = np.clip(decorated_aligned, 0, vmax)
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(131)
    plt.imshow(decorated_image)
    plt.title("Decorated RGB")
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(decorated_viz, cmap='magma', vmin=0, vmax=vmax)
    plt.title(f"Decorated Depth (×{scale:.5f})")
    plt.colorbar(shrink=0.8)
    plt.axis('off')
    
    plt.subplot(133)
    # Resize to GT res just for this diff comparison
    dec_aligned_resized = cv2.resize(decorated_aligned, (w_gt, h_gt), interpolation=cv2.INTER_LINEAR)
    depth_diff = np.abs(dec_aligned_resized - pred_aligned)
    plt.imshow(depth_diff, cmap='viridis')
    plt.title("Depth Difference (Decorated - Original)")
    plt.colorbar(shrink=0.8)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()



In [ ]:
# =========================================
# 1. PREPARE VISUALIZATION DATA
# =========================================
# Create error maps
abs_diff = np.abs(gt_depth - pred_aligned)
signed_diff = pred_aligned - gt_depth
rel_diff = abs_diff / (gt_depth + 1e-6)

# Mask out invalid pixels for cleaner plots
abs_diff[~valid_mask] = 0
signed_diff[~valid_mask] = 0
rel_diff[~valid_mask] = 0

# Set dynamic limits based on percentiles (ignores outliers)
depth_vmax = np.percentile(gt_depth[valid_mask], 95)
err_vmax = np.percentile(abs_diff[valid_mask], 95)
signed_vmax = max(np.percentile(np.abs(signed_diff[valid_mask]), 95), 1e-3)

# =========================================
# 2. MAIN DASHBOARD (2 Rows x 4 Columns)
# =========================================
plt.figure(figsize=(24, 12))

# --- ROW 1: Qualitative (Visuals) ---

# 1. Input RGB
plt.subplot(2, 4, 1)
plt.imshow(image)
plt.title("Input RGB", fontsize=14, fontweight='bold')
plt.axis('off')

# 2. Ground Truth
plt.subplot(2, 4, 2)
plt.imshow(gt_depth, cmap='magma', vmin=0, vmax=depth_vmax)
plt.title("Ground Truth Depth", fontsize=14, fontweight='bold')
plt.colorbar(label="Depth", shrink=0.6)
plt.axis('off')

# 3. Prediction
plt.subplot(2, 4, 3)
plt.imshow(pred_aligned, cmap='magma', vmin=0, vmax=depth_vmax)
plt.title(f"Predicted Depth\n(Scaled x{scale:.4f})", fontsize=14, fontweight='bold')
plt.colorbar(label="Depth", shrink=0.6)
plt.axis('off')

# 4. Absolute Error (Heatmap)
plt.subplot(2, 4, 4)
plt.imshow(abs_diff, cmap='inferno', vmin=0, vmax=err_vmax)
plt.title(f"Absolute Error\n(MAE: {mae:.4f})", fontsize=14, fontweight='bold')
plt.colorbar(label="Abs Diff", shrink=0.6)
plt.axis('off')

# --- ROW 2: Quantitative (Analytics) ---

# 5. Signed Error (Bias Check)
# Red = Prediction is too far, Blue = Prediction is too close
plt.subplot(2, 4, 5)
plt.imshow(signed_diff, cmap='RdBu_r', vmin=-signed_vmax, vmax=signed_vmax)
plt.title("Signed Error (Bias)\nRed = Overest., Blue = Underest.", fontsize=14, fontweight='bold')
plt.colorbar(label="Pred - GT", shrink=0.6)
plt.axis('off')

# 6. Relative Error
plt.subplot(2, 4, 6)
plt.imshow(rel_diff, cmap='Reds', vmin=0, vmax=0.5) # Cap at 50% error for visibility
plt.title(f"Relative Error\n(AbsRel: {rel_error:.2f}%)", fontsize=14, fontweight='bold')
plt.colorbar(label="Rel Error", shrink=0.6)
plt.axis('off')

# 7. Error Histogram
plt.subplot(2, 4, 7)
plt.hist(abs_diff[valid_mask], bins=50, range=(0, err_vmax), color='steelblue', alpha=0.7, edgecolor='black')
plt.xlabel("Absolute Error")
plt.ylabel("Pixel Count")
plt.title("Error Distribution", fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# 8. Metrics Summary Box
plt.subplot(2, 4, 8)
plt.axis('off')
metrics_text = (
    f"MODEL METRICS\n"
    f"----------------------\n"
    f"RMSE:     {rmse:.4f}\n"
    f"MAE:      {mae:.4f}\n"
    f"AbsRel:   {rel_error:.2f}%\n"
    f"----------------------\n"
    f"ACCURACY (δ)\n"
    f"δ < 1.25: {delta1*100:.2f}%\n"
    f"----------------------\n"
    f"SCALING\n"
    f"Method:   {SCALING_METHOD}\n"
    f"Scale:    {scale:.4f}"
)
plt.text(0.5, 0.5, metrics_text, fontsize=16, family='monospace', va='center', ha='center',
         bbox=dict(boxstyle="round,pad=1", facecolor='whitesmoke', edgecolor='gray', alpha=0.5))

plt.tight_layout()
plt.show()

# =========================================
# 3. DECORATED COMPARISON (Keep as separate block)
# =========================================
if decorated_depth is not None:
    decorated_aligned = decorated_depth * scale
    
    # Resize decorated to GT size for accurate diff
    dec_aligned_resized = cv2.resize(decorated_aligned, (w_gt, h_gt), interpolation=cv2.INTER_LINEAR)
    
    # Difference: How much did the decoration change the depth?
    change_map = dec_aligned_resized - pred_aligned
    change_vmax = max(np.percentile(np.abs(change_map), 99), 1e-3)

    plt.figure(figsize=(18, 6))
    
    plt.subplot(131)
    plt.imshow(decorated_image)
    plt.title("Decorated Input", fontsize=12, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(decorated_aligned, cmap='magma', vmin=0, vmax=depth_vmax)
    plt.title("Decorated Depth Prediction", fontsize=12, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(change_map, cmap='RdBu_r', vmin=-change_vmax, vmax=change_vmax)
    plt.title("Impact of Decoration\n(Decorated - Original)", fontsize=12, fontweight='bold')
    plt.colorbar(label="Depth Change", shrink=0.8)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
print("\nGenerating point cloud...")

# Convert depth to centimeters then meters
depth_cm = pred_aligned * GT_TO_CENTIMETERS   # already multiplied by scale + shift
depth_m = depth_cm / 100.0

print(f"Depth range: {depth_m.min():.2f}m to {depth_m.max():.2f}m (mean: {depth_m.mean():.2f}m)")

# Compute focal length from FOV
h, w = depth_m.shape
focal_length = w / (2 * np.tan(np.radians(CAMERA_FOV / 2)))
print(f"Focal length from FOV ({CAMERA_FOV}°): {focal_length:.2f}px")

# Back-project to 3D
cx, cy = w / 2, h / 2
xx, yy = np.meshgrid(np.arange(w), np.arange(h))

z = depth_m.flatten()
x = (xx.flatten() - cx) * z / focal_length
y = (yy.flatten() - cy) * z / focal_length

# Get colors
rgb = np.array(image.resize((w, h))).reshape(-1, 3)

# Filter by depth range
mask = (z >= MIN_DEPTH) & (z <= MAX_DEPTH) & (z > 0)

# Optional edge filtering
if CLEAN_POINT_CLOUD:
    print("Applying edge filtering...")
    depth_grad_x = cv2.Sobel(depth_m, cv2.CV_64F, 1, 0, ksize=3)
    depth_grad_y = cv2.Sobel(depth_m, cv2.CV_64F, 0, 1, ksize=3)
    grad_mag = np.sqrt(depth_grad_x**2 + depth_grad_y**2)
    edge_mask = (grad_mag < EDGE_THRESHOLD).flatten()
    mask = mask & edge_mask

x, y, z, rgb = x[mask], y[mask], z[mask], rgb[mask]

print(f"✓ Generated {len(z):,} points")


# ← NEW: Generate decorated point cloud if available
if decorated_depth is not None:
    print("\nGenerating decorated point cloud...")
    
    # Convert depth to centimeters then meters (use same scale)
    depth_cm_dec = (decorated_depth * scale + shift) * GT_TO_CENTIMETERS   # shift 0 for median
    depth_m_dec = depth_cm_dec / 100.0
    
    print(f"Decorated depth range: {depth_m_dec.min():.2f}m to {depth_m_dec.max():.2f}m (mean: {depth_m_dec.mean():.2f}m)")

    # Recompute grid at decorated image's own resolution
    h_dec, w_dec = depth_m_dec.shape
    focal_length_dec = w_dec / (2 * np.tan(np.radians(CAMERA_FOV / 2)))
    cx_dec, cy_dec = w_dec / 2, h_dec / 2
    xx_dec, yy_dec = np.meshgrid(np.arange(w_dec), np.arange(h_dec))
    
    # Back-project to 3D
    z_dec = depth_m_dec.flatten()
    x_dec = (xx_dec.flatten() - cx_dec) * z_dec / focal_length_dec
    y_dec = (yy_dec.flatten() - cy_dec) * z_dec / focal_length_dec
    
    # Get colors from decorated image
    rgb_dec = np.array(decorated_image).reshape(-1, 3)
    
    # Filter by depth range
    mask_dec = (z_dec >= MIN_DEPTH) & (z_dec <= MAX_DEPTH) & (z_dec > 0)
    
    # Optional edge filtering
    if CLEAN_POINT_CLOUD:
        print("Applying edge filtering to decorated...")
        depth_grad_x_dec = cv2.Sobel(depth_m_dec, cv2.CV_64F, 1, 0, ksize=3)
        depth_grad_y_dec = cv2.Sobel(depth_m_dec, cv2.CV_64F, 0, 1, ksize=3)
        grad_mag_dec = np.sqrt(depth_grad_x_dec**2 + depth_grad_y_dec**2)
        edge_mask_dec = (grad_mag_dec < EDGE_THRESHOLD).flatten()
        mask_dec = mask_dec & edge_mask_dec
    
    x_dec, y_dec, z_dec, rgb_dec = x_dec[mask_dec], y_dec[mask_dec], z_dec[mask_dec], rgb_dec[mask_dec]
    
    print(f"✓ Generated {len(z_dec):,} decorated points")

In [ ]:
print("\nSaving point cloud...")

header = laspy.LasHeader(point_format=3, version="1.2")
header.scales = np.array([0.001, 0.001, 0.001])
las = laspy.LasData(header=header)

# LAS coordinate system: Z forward, -X right, -Y up
las.x = z
las.y = -x
las.z = -y
las.red = rgb[:, 0].astype(np.uint16) * 256
las.green = rgb[:, 1].astype(np.uint16) * 256
las.blue = rgb[:, 2].astype(np.uint16) * 256

las.write(OUTPUT_LAS)
print(f"✓ Saved {len(z):,} points to {OUTPUT_LAS}")


# ← NEW: Save decorated point cloud if available
if decorated_depth is not None:
    print("\nSaving decorated point cloud...")
    
    header_dec = laspy.LasHeader(point_format=3, version="1.2")
    header_dec.scales = np.array([0.001, 0.001, 0.001])
    las_dec = laspy.LasData(header_dec)
    
    # LAS coordinate system: Z forward, -X right, -Y up
    las_dec.x = z_dec
    las_dec.y = -x_dec
    las_dec.z = -y_dec
    las_dec.red = rgb_dec[:, 0].astype(np.uint16) * 256
    las_dec.green = rgb_dec[:, 1].astype(np.uint16) * 256
    las_dec.blue = rgb_dec[:, 2].astype(np.uint16) * 256
    
    las_dec.write(OUTPUT_LAS_DECORATED)
    print(f"✓ Saved {len(z_dec):,} decorated points to {OUTPUT_LAS_DECORATED}")
